## Applicative

### Applicative definition

In Haskell, the `Applicative` type class is a powerful abstraction that allows you to apply functions wrapped in a context (like a `Maybe`, `List`, or `IO`) to values that are also wrapped in a context. It builds on the `Functor` type class, and provides a way to handle functions that take multiple arguments in a context-aware manner.

#### Key Components

1. **Functor**: The `Applicative` type class is a subclass of `Functor`. This means that any type that is an instance of `Applicative` must also implement `fmap`.

2. **pure**: This function takes a value and wraps it in the applicative context.

3. **(<*> or ap)**: This operator takes a wrapped function and a wrapped value and applies the function to the value.

#### Type Class Definition

Here’s a simplified definition of the `Applicative` type class:

```haskell
class Functor f => Applicative f where
    pure :: a -> f a
    (<*>) :: f (a -> b) -> f a -> f b
```

#### Example 1: `Maybe`

The `Maybe` type is a common instance of `Applicative`. It represents a computation that might fail.

```haskell
-- Using pure to wrap a value
justFive :: Maybe Int
justFive = pure 5  -- Result: Just 5

-- Applying a function wrapped in Maybe
add :: Int -> Int -> Int
add x y = x + y

maybeAdd :: Maybe (Int -> Int)
maybeAdd = pure add

result :: Maybe Int
result = maybeAdd <*> Just 3 <*> Just 4  -- Result: Just 7
```

#### Example 2: Lists

Lists are also instances of `Applicative`. The `(<*>)` operator applies functions in a list to values in another list, producing all combinations.

```haskell
-- Pure to wrap a value in a list
pureList :: [Int]
pureList = pure 5  -- Result: [5]

-- Using a list of functions
addLists :: [Int -> Int]
addLists = [(+1), (+2)]

-- Combining lists
resultList :: [Int]
resultList = addLists <*> [1, 2, 3]  -- Result: [2,3,4,3,4,5]
```

#### Example 3: `IO`

The `IO` type is also an instance of `Applicative`, allowing you to combine actions.

```haskell
-- Pure to wrap a value in IO
pureIO :: IO Int
pureIO = pure 5  -- Result: an IO action that produces 5

-- Combining IO actions
printSum :: IO ()
printSum = (+) <$> getLine <*> getLine >>= print
```

### Summary

- `Applicative` allows you to apply functions in a context to values in a context.
- It is a generalization of `Functor` and provides more functionality.
- Common instances include `Maybe`, lists, and `IO`.

By using `Applicative`, you can elegantly handle computations that may involve multiple contexts, making your Haskell code more concise and expressive.